In [1]:
!python make_dataset.py
!python train_model.py

[{'len': 900, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 900,
  'n_features': 10,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]
[{'len': 100, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 100,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]


The system cannot find the path specified.


[{'name': '<cascade.utils.sk_model.SkModel object at 0x00000229AABDDAF0>', 'created_at': DateTime(2022, 12, 4, 11, 27, 53, 17143, tzinfo=Timezone('UTC')), 'metrics': {'f1': 0.9967141292442497}, 'params': {}, 'type': 'model', 'pipeline': "Pipeline(steps=[('0', SVC(probability=True))])"}]


The system cannot find the path specified.


In [2]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd

In [3]:
from correctness import correctness
from continuity import continuity
from contrastivity import contrastivity
from coherence import coherence

c:\xaib-experiments\venv3.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from xaib.explainers.feature_importance.constant_explainer import ConstantExplainer
from xaib.explainers.feature_importance.random_explainer import RandomExplainer
from xaib.explainers.feature_importance.shap_explainer import ShapExplainer
from xaib.explainers.feature_importance.lime_explainer import LimeExplainer

In [5]:
BS = 100

In [6]:
# Overwrite previous run
ModelRepo('repo', overwrite=True)

ModelRepo in repo of 0 lines

In [7]:
train_ds = cdd.Pickler('train_ds').ds()
n_features = train_ds.get_meta()[0]['n_features']

In [8]:
explainers = {
    'const': ConstantExplainer(n_features=n_features, constant=1),
    'random': RandomExplainer(n_features=n_features, shift=-15, magnitude=10),
    'shap': ShapExplainer(train_ds),
    'lime': LimeExplainer(train_ds, labels=(0, 1))
}

In [9]:
correctness(explainers, batch_size=BS)

100%|██████████| 1/1 [00:00<00:00, 111.12it/s]
Exact explainer: 101it [03:27,  2.08s/it]
Exact explainer: 101it [02:41,  1.72s/it]
100%|██████████| 1/1 [00:46<00:00, 46.79s/it]


In [10]:
continuity(explainers, batch_size=BS)

1it [00:00, 332.78it/s]
1it [00:00, 200.26it/s]
Exact explainer: 101it [03:19,  2.02s/it]
Exact explainer: 101it [02:52,  1.82s/it]
1it [06:12, 372.36s/it]
1it [00:52, 52.90s/it]


In [11]:
contrastivity(explainers, batch_size=BS)

100%|██████████| 1/1 [00:00<00:00, 501.29it/s]
Exact explainer: 45it [01:34,  2.19s/it]                 
100%|██████████| 1/1 [01:34<00:00, 94.14s/it]
Exact explainer: 45it [01:13,  1.93s/it]
100%|██████████| 1/1 [00:13<00:00, 13.70s/it]


In [12]:
coherence(explainers, batch_size=BS)

Exact explainer: 101it [03:11,  1.94s/it]
100%|██████████| 1/1 [03:27<00:00, 207.38s/it]
Exact explainer: 101it [03:17,  2.00s/it]
100%|██████████| 1/1 [03:32<00:00, 212.60s/it]
Exact explainer: 101it [03:16,  1.99s/it]
100%|██████████| 1/1 [03:32<00:00, 212.57s/it]
Exact explainer: 101it [02:48,  1.79s/it]
100%|██████████| 1/1 [03:27<00:00, 207.09s/it]


In [13]:
repo = ModelRepo('repo')

In [14]:
t = MetricViewer(repo).table

In [15]:
def present_metrics(case):
    return t[t['line'].apply(lambda x: case in x)].dropna(axis=1)

In [16]:
present_metrics('correctness')

,line,num,created_at,saved,name,parameter_randomization_check
12,repo\correctness,0,2022-12-04 11:28:02.015009+00:00,a few seconds after,const,0.000000
13,repo\correctness,1,2022-12-04 11:28:02.015009+00:00,a few seconds after,random,1.289212
14,repo\correctness,2,2022-12-04 11:28:02.015009+00:00,6 minutes after,shap,0.593295
15,repo\correctness,3,2022-12-04 11:28:02.015009+00:00,6 minutes after,lime,0.678844


In [17]:
present_metrics('continuity')

,line,num,created_at,saved,name,small_noise_check,multiplier
4,repo\continuity,0,2022-12-04 11:34:58.920351+00:00,a few seconds after,const,0.000000,0.01
5,repo\continuity,1,2022-12-04 11:34:58.920351+00:00,a few seconds after,random,1.285975,0.01
6,repo\continuity,2,2022-12-04 11:34:58.920351+00:00,6 minutes after,shap,0.001257,0.01
7,repo\continuity,3,2022-12-04 11:34:58.920351+00:00,7 minutes after,lime,0.062066,0.01


In [18]:
present_metrics('contrastivity')

,line,num,created_at,saved,name,label_difference
8,repo\contrastivity,0,2022-12-04 11:42:04.313409+00:00,a few seconds after,const,0.000000
9,repo\contrastivity,1,2022-12-04 11:42:04.313409+00:00,a few seconds after,random,1.247543
10,repo\contrastivity,2,2022-12-04 11:42:04.313409+00:00,2 minutes after,shap,1.714009
11,repo\contrastivity,3,2022-12-04 11:42:04.313409+00:00,3 minutes after,lime,0.502633


In [19]:
present_metrics('coherence')

,line,num,created_at,saved,other_disagreement,name
0,repo\coherence,0,2022-12-04 11:45:29.427824+00:00,3 minutes after,1.712317,const
1,repo\coherence,1,2022-12-04 11:45:29.427824+00:00,7 minutes after,1.273758,random
2,repo\coherence,2,2022-12-04 11:45:29.427824+00:00,10 minutes after,1.105160,shap
3,repo\coherence,3,2022-12-04 11:45:29.427824+00:00,13 minutes after,1.004815,lime
